In [61]:
import numpy as np
from lava.proc.lif.process import LIF
from lava.proc.monitor.process import Monitor
from lava.proc.monitor.models import PyMonitorModel
from lava.proc.dense.process import Dense
from lava.proc.io.sink import RingBuffer
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights, Convolution
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.rate_code_spike_gen.process import RateCodeSpikeGen
from lava.lib.dnf.kernels.kernels import Kernel
#from utils import plot_1d, animated_1d_plot

Initialize the plug plate event camera input signal

In [62]:
shape = (10, 10)


# instancite Spikegenerator (plug plate)
spike_generator = RateCodeSpikeGen(shape=shape)


In [63]:
class Architecture:

    def __init__(self,kernel: Kernel) -> None:
        
        n = 10
        shape = (n, n)
        time_steps = 700
        
        # Instantiate Processes
        self.gauss_pattern_1 = GaussPattern(shape=shape,
                                            amplitude=10,
                                            mean=[1, 1],
                                            stddev=[2.25, 2.25])
        self.gauss_pattern_2 = GaussPattern(shape=shape,
                                            amplitude=10,
                                            mean=[1, 8],
                                            stddev=[2.25, 2.25])
        self.gauss_pattern_3 = GaussPattern(shape=shape,
                                            amplitude=5,
                                            mean=[8, 1],
                                            stddev=[2.25, 2.25])
        self.gauss_pattern_4 = GaussPattern(shape=shape,
                                            amplitude=5,
                                            mean=[8, 8],
                                            stddev=[2.25, 2.25])
        
        self.spike_generator = RateCodeSpikeGen(shape=shape) 
        self.input_dense = Dense(weights=np.ones(shape))

        
        self.dnf = LIF(shape=shape, du=409, dv=2047, vth=200)
        self.py_receiver = RingBuffer(shape=shape, buffer=time_steps)
        
        # When running on Loihi 2 we additionally have spike injector and reader CProcesses. 
        # They bridge Python and NC-Processes
        # if loihi2_is_available:
        #    self.injector1 = PyToNxAdapter(shape=shape)
        #    self.injector2 = PyToNxAdapter(shape=shape)
        #    self.spike_reader = NxToPyAdapter(shape=shape)
        
        # Make Connections of the Network
        self.gauss_pattern_1.a_out.connect(self.spike_generator.a_in)
        #self.gauss_pattern_2.a_out.connect(self.spike_generator.a_in)
        #self.gauss_pattern_3.a_out.connect(self.spike_generator.a_in)
        #self.gauss_pattern_4.a_out.connect(self.spike_generator.a_in)
        

        
        self.spike_generator.s_out.connect(self.input_dense.s_in)
        self.input_dense.a_out.connect(self.dnf.a_in)        
        connect(self.dnf.s_out, self.dnf.a_in, [Convolution(kernel)])
        
        self.dnf.s_out.connect(self.py_receiver.a_in)
        
        
        # Set up monitors
        self.monitor_input_1 = Monitor()
        self.monitor_input_1.probe(self.spike_generator_1.s_out, time_steps)


        # Set up a run configuration
        self.run_cfg = Loihi1SimCfg(select_tag="fixed_pt")
        

    def run(self):
        condition = RunSteps(num_steps=100)
        self.gauss_pattern_1.run(condition=condition, run_cfg=self.run_cfg)
        self.gauss_pattern_1.amplitude = 2300
        #self.gauss_pattern_2.amplitude = 2300
        self.gauss_pattern_1.run(condition=condition, run_cfg=self.run_cfg)
        self.gauss_pattern_1.amplitude = 11200
        #self.gauss_pattern_2.amplitude = 11200
        self.gauss_pattern_1.run(condition=condition, run_cfg=self.run_cfg)
        self.gauss_pattern_1.amplitude = 2300
        #self.gauss_pattern_2.amplitude = 2300
        #self.gauss_pattern_1.run(condition=RunSteps(num_steps=200),
        #                         run_cfg=self.run_cfg)
        self.gauss_pattern_1.amplitude = 0
        #self.gauss_pattern_2.amplitude = 0
        self.gauss_pattern_1.run(condition=RunSteps(num_steps=200),
                                 run_cfg=self.run_cfg)


    def plot(self):
        # Get probed data from monitors
        data_dnf = self.py_receiver.data.get().transpose()
        data_input1 = self.monitor_input_1.get_data() \
            [self.spike_generator.name][self.spike_generator.s_out.name]
    
        # Stop the execution of the network
        self.dnf.stop()

        # Generate a raster plot from the probed data
        plot_1d(data_dnf,
                data_input1,
                data_input2)

        # Generate an animated plot from the probed data
        animated_1d_plot(data_dnf,
                         data_input1,
                         data_input2)

In [64]:
from lava.lib.dnf.kernels.kernels import MultiPeakKernel

detection_kernel = MultiPeakKernel(amp_exc=83, 
                                   width_exc=[3.75, 3.75], 
                                   amp_inh=-70,
                                   width_inh=[7.5, 7.5])

architecture = Architecture(detection_kernel)

#architecture.run()
#architecture.plot()

AttributeError: 'Dense' object has no attribute 'shape'

In [65]:
architecture

NameError: name 'architecture' is not defined

In [ ]:
self.spike_generator


kernel = MultiPeakKernel(amp_exc=58,
                         width_exc=[3.8, 3.8],
                         amp_inh=-50,
                         width_inh=[7.5, 7.5])

self.spike_generator.s_out.connect(dnf.a_in)
connect(dnf.s_out, dnf.a_in, ops=[Convolution(kernel)])